In [72]:
# 简单神经网络实现任意32*32维矩阵到0~9的转化
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

#1. 构建LeNet
class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        # 下式等价于nn.Module.__init__(self)
        super(Net, self).__init__()
        
        # 卷积层‘１’表示输入图片为单通道，‘６’表示输出通道数
        # ‘５’表示卷积核为５＊５
        self.conv1 = nn.Conv2d(1, 6, 5)
        
        # 卷积层
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # 仿射层／全连接层，y=Wx+b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # 顺序表示运行顺序，前馈经过顺序
        # 卷积　－> 激活（激活函数relu）　－> 池化（pool采样）
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))# s2
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)# s4
        
        # reshape, '-1' 表示自适应
        x = x.view(x.size()[0], -1)# 16个5*5不改变数据的情况下，自适应到400
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

#2. 输出Net
net = Net()
print(net)

#3. 输出参数
params = list(net.parameters())
print(len(params))

# 参数，权重矩阵和向上传递的偏差矩阵
for name, parameters in net.named_parameters():
    print(name, ':', parameters.size())
#４．训练
optimizer = optim.SGD(net.parameters(), lr = 0.01)# 设置学习率
for i in range(10000):
    #4.1 前向传播
    input = Variable(t.randn(1, 1, 32, 32))# 定义输入，这里只用了一个案例输入一个
    output = net(input)# 进行一次前向传播

    #4.2 计算损失(损失函数)
    target = Variable(t.arange(0, 10))
    criterion = nn.MSELoss()
    loss = criterion(output, target)

    #4.3 反向传播计算损失
    net.zero_grad()# 可学习参数清零
    loss.backward()# 直接反向传播loss即可
    
    #4.4 更新参数
    optimizer.step()

#5. 测试
input = Variable(t.randn(1, 1, 32, 32))# 定义输入，这里只用了一个案例输入一个
output = net(input)# 进行一次前向传播
print(output)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)
10
conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([84, 120])
fc2.bias : torch.Size([84])
fc3.weight : torch.Size([10, 84])
fc3.bias : torch.Size([10])
Variable containing:
-0.0007  0.9947  1.9911  2.9881  3.9821  4.9802  5.9743  6.9712  7.9689  8.9650
[torch.FloatTensor of size 1x10]

